# Alphafold 3 결과 확인

SEIHWAN 2025

In [ ]:
# 필요한 라이브러리 설치
!pip -q install py3Dmol ipywidgets -q

# 필요한 라이브러리 임포트
import os
import zipfile
import py3Dmol
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# 파일 업로드 및 처리 함수
def upload_and_extract():
    print("AlphaFold 3 결과 압축 파일을 업로드하세요:")
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return []

    # 압축 파일 경로
    zip_path = list(uploaded.keys())[0]
    print(f"업로드된 파일: {zip_path}")

    # 압축 해제할 디렉토리
    output_dir = '/content/alphafold_unpacked'
    if os.path.exists(output_dir):
        import shutil
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
        print(f"파일이 {output_dir}에 해제되었습니다.")

    # 구조 파일 찾기
    structure_files = []
    for root, dirs, files_list in os.walk(output_dir):
        for file in files_list:
            if file.endswith('.pdb') or file.endswith('.cif'):
                structure_files.append(os.path.join(root, file))

    if not structure_files:
        print("구조 파일(.pdb 또는 .cif)을 찾을 수 없습니다.")
        # 다른 가능한 확장자 찾기
        for root, dirs, files_list in os.walk(output_dir):
            for file in files_list:
                try:
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as f:
                        first_line = f.readline()
                        if 'ATOM' in first_line or 'HEADER' in first_line:
                            structure_files.append(file_path)
                            print(f"가능한 구조 파일 발견: {file_path}")
                except:
                    pass

    print(f"\n발견된 구조 파일 수: {len(structure_files)}")
    return structure_files

# 구조 시각화 함수
def visualize_structure(file_path, style='cartoon'):
    """구조 파일을 시각화합니다."""
    clear_output(wait=True)
    print(f"파일 로딩 중: {os.path.basename(file_path)}, 스타일: {style}")

    # 파일 형식 확인 (확장자 기반)
    file_format = 'pdb'  # 기본값
    if file_path.endswith('.cif'):
        file_format = 'cif'

    try:
        # 파일 내용 읽기
        with open(file_path, 'r') as f:
            structure_data = f.read()

        # 구조 시각화
        view = py3Dmol.view(width=800, height=500)
        view.addModel(structure_data, file_format)

        # 스타일 적용
        if style == 'cartoon':
            view.setStyle({'cartoon': {'color': 'spectrum'}})
        elif style == 'stick':
            view.setStyle({'stick': {}})
        elif style == 'sphere':
            view.setStyle({'sphere': {'radius': 0.5}})
        elif style == 'line':
            view.setStyle({'line': {}})
        elif style == 'ribbon':
            # 리본 스타일 수정 - 카툰 대신 직접 리본 설정
            view.setStyle({}, {'cartoon': {'style': 'ribbon', 'color': 'spectrum'}})
        else:
            view.setStyle({'cartoon': {'color': 'spectrum'}})

        view.zoomTo()
        view.spin(True)
        display(view.show())

        print(f"\n구조 시각화 완료: {os.path.basename(file_path)}")
        print("파일이나 스타일을 변경하려면 아래 버튼을 사용하세요.")

    except Exception as e:
        print(f"오류 발생: {str(e)}")

# 메인 기능 실행
structure_files = upload_and_extract()

if structure_files:
    # 파일 선택 버튼 생성
    file_buttons = []
    for i, file_path in enumerate(structure_files):
        file_name = os.path.basename(file_path)
        button = widgets.Button(
            description=file_name[:20] + ('...' if len(file_name) > 20 else ''),
            tooltip=file_name,
            layout=widgets.Layout(width='auto', height='auto')
        )
        # 더 진한 회색으로 변경
        button.style.button_color = '#A0A0A0'  # 진한 회색
        button.style.text_color = 'black'
        button.file_path = file_path  # 파일 경로 저장
        file_buttons.append(button)

    # 스타일 선택 버튼 생성
    style_buttons = []
    styles = ['cartoon', 'ribbon', 'stick', 'sphere', 'line']  # 리본을 두 번째로 이동
    style_colors = {
        'cartoon': '#4682B4',  # 스틸 블루
        'ribbon': '#556B2F',    # 다크 올리브 그린 (리본)
        'stick': '#8B4513',    # 새들 브라운
        'sphere': '#6A5ACD',   # 슬레이트 블루
        'line': '#2F4F4F',     # 다크 슬레이트 그레이
    }

    for style in styles:
        button = widgets.Button(
            description=style,
            layout=widgets.Layout(width='100px', height='auto')
        )
        button.style.button_color = style_colors[style]
        button.style.text_color = 'white'  # 어두운 배경에 흰색 글씨
        button.style_name = style  # 스타일 이름 저장
        style_buttons.append(button)

    # 현재 선택된 파일과 스타일 저장
    state = {'current_file': structure_files[0], 'current_style': 'cartoon'}

    # 버튼 클릭 이벤트 핸들러
    def on_file_button_clicked(b):
        state['current_file'] = b.file_path
        # 모든 파일 버튼 색상 초기화
        for button in file_buttons:
            button.style.button_color = '#A0A0A0'  # 진한 회색
            button.style.text_color = 'black'
        # 선택된 버튼 색상 변경
        b.style.button_color = '#228B22'  # 포레스트 그린 (더 진한 녹색)
        b.style.text_color = 'white'
        # 구조 시각화
        visualize_structure(state['current_file'], state['current_style'])
        # 버튼 다시 표시
        display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
        display(widgets.HBox(file_buttons))
        display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
        display(widgets.HBox(style_buttons))

    def on_style_button_clicked(b):
        state['current_style'] = b.style_name
        # 모든 스타일 버튼 색상 변경 (각 버튼마다 고유한 색상)
        for button in style_buttons:
            style = button.style_name
            if style == state['current_style']:
                # 선택된 버튼은 더 밝은 버전의 색상
                brightness_factor = 0.7
                r = int(int(style_colors[style][1:3], 16) * (1 + brightness_factor))
                g = int(int(style_colors[style][3:5], 16) * (1 + brightness_factor))
                b = int(int(style_colors[style][5:7], 16) * (1 + brightness_factor))
                r = min(r, 255)
                g = min(g, 255)
                b = min(b, 255)
                button.style.button_color = f'#{r:02x}{g:02x}{b:02x}'
            else:
                button.style.button_color = style_colors[style]
            button.style.text_color = 'white'

        # 구조 시각화
        visualize_structure(state['current_file'], state['current_style'])
        # 버튼 다시 표시
        display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
        display(widgets.HBox(file_buttons))
        display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
        display(widgets.HBox(style_buttons))

    # 버튼에 이벤트 핸들러 연결
    for button in file_buttons:
        button.on_click(on_file_button_clicked)

    for button in style_buttons:
        button.on_click(on_style_button_clicked)

    # 첫 번째 파일 자동 표시
    visualize_structure(structure_files[0], 'cartoon')

    # 버튼 표시
    display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
    display(widgets.HBox(file_buttons))
    display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
    display(widgets.HBox(style_buttons))

    # 첫 번째 파일 버튼 하이라이트
    file_buttons[0].style.button_color = '#228B22'  # 포레스트 그린
    file_buttons[0].style.text_color = 'white'

    # 카툰 스타일 버튼 하이라이트
    brightness_factor = 0.7
    r = int(int(style_colors['cartoon'][1:3], 16) * (1 + brightness_factor))
    g = int(int(style_colors['cartoon'][3:5], 16) * (1 + brightness_factor))
    b = int(int(style_colors['cartoon'][5:7], 16) * (1 + brightness_factor))
    r = min(r, 255)
    g = min(g, 255)
    b = min(b, 255)
    style_buttons[0].style.button_color = f'#{r:02x}{g:02x}{b:02x}'
    style_buttons[0].style.text_color = 'white'

파일 로딩 중: examplefold_pdb_7rce_model_2.cif, 스타일: cartoon


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

None


구조 시각화 완료: examplefold_pdb_7rce_model_2.cif
파일이나 스타일을 변경하려면 아래 버튼을 사용하세요.


HTML(value='<h3>구조 파일 선택:</h3>')

HTML(value='<h3>시각화 스타일 선택:</h3>')

# Diffdock web 결과 확인

In [ ]:
# 필요한 라이브러리 설치
!pip -q install py3Dmol ipywidgets -q

# 필요한 라이브러리 임포트
import os
import zipfile
import py3Dmol
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# 파일 업로드 및 처리 함수
def upload_and_extract():
    print("DiffDock 결과 압축 파일을 업로드하세요:")
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return []

    # 압축 파일 경로
    zip_path = list(uploaded.keys())[0]
    print(f"업로드된 파일: {zip_path}")

    # 압축 해제할 디렉토리
    output_dir = '/content/diffdock_unpacked'
    if os.path.exists(output_dir):
        import shutil
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
        print(f"파일이 {output_dir}에 해제되었습니다.")

    # 구조 파일 찾기 (DiffDock은 complex 폴더 안에 PDB 파일이 있음)
    structure_files = []
    for root, dirs, files_list in os.walk(output_dir):
        for file in files_list:
            if file.endswith('.pdb') and 'complex' in root:
                structure_files.append(os.path.join(root, file))

    if not structure_files:
        print("complex 폴더 안에 PDB 파일을 찾을 수 없습니다.")
        # 다른 가능한 위치에서 PDB 파일 찾기
        for root, dirs, files_list in os.walk(output_dir):
            for file in files_list:
                if file.endswith('.pdb'):
                    structure_files.append(os.path.join(root, file))
                    print(f"다른 위치에서 PDB 파일 발견: {os.path.join(root, file)}")

    # 로그 파일 찾기 및 표시
    log_files = []
    for root, dirs, files_list in os.walk(output_dir):
        for file in files_list:
            if file.endswith('.log'):
                log_files.append(os.path.join(root, file))
                print(f"로그 파일 발견: {os.path.join(root, file)}")

    print(f"\n발견된 구조 파일 수: {len(structure_files)}")
    return structure_files, log_files

# 구조 시각화 함수
def visualize_structure(file_path, style='cartoon'):
    """구조 파일을 시각화합니다."""
    clear_output(wait=True)
    print(f"파일 로딩 중: {os.path.basename(file_path)}, 스타일: {style}")

    # 파일 형식 확인 (확장자 기반)
    file_format = 'pdb'  # 기본값
    if file_path.endswith('.cif'):
        file_format = 'cif'

    try:
        # 파일 내용 읽기
        with open(file_path, 'r') as f:
            structure_data = f.read()

        # 구조 시각화
        view = py3Dmol.view(width=800, height=500)
        view.addModel(structure_data, file_format)

        # 스타일 적용
        if style == 'cartoon':
            view.setStyle({'cartoon': {'color': 'spectrum'}})
        elif style == 'stick':
            view.setStyle({'stick': {}})
        elif style == 'sphere':
            view.setStyle({'sphere': {'radius': 0.5}})
        elif style == 'line':
            view.setStyle({'line': {}})
        elif style == 'ribbon':
            # 리본 스타일 수정 - 카툰 대신 직접 리본 설정
            view.setStyle({}, {'cartoon': {'style': 'ribbon', 'color': 'spectrum'}})
        elif style == 'surface':
            view.setStyle({'surface': {'opacity': 0.8}})
        elif style == 'ligand_highlight':
            # 리간드(HET) 강조 스타일 - 단백질은 카툰, 리간드는 스틱
            view.setStyle({'cartoon': {'color': 'spectrum'}})
            view.setStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon', 'radius': 0.2}})
        else:
            view.setStyle({'cartoon': {'color': 'spectrum'}})

        # 단백질과 리간드 경계 강조
        view.addSurface(py3Dmol.VDW, {'opacity': 0.6, 'color': 'white'}, {'hetflag': True})

        view.zoomTo()
        view.spin(True)
        display(view.show())

        print(f"\n구조 시각화 완료: {os.path.basename(file_path)}")
        print("파일이나 스타일을 변경하려면 아래 버튼을 사용하세요.")

    except Exception as e:
        print(f"오류 발생: {str(e)}")

# 로그 파일 보기 함수
def view_log_file(log_path):
    """로그 파일 내용을 표시합니다."""
    try:
        with open(log_path, 'r') as f:
            log_content = f.read()

        output = widgets.Output()
        with output:
            print(f"로그 파일: {os.path.basename(log_path)}")
            print("-" * 80)
            print(log_content)

        return output
    except Exception as e:
        print(f"로그 파일 읽기 오류: {str(e)}")
        return None

# 메인 기능 실행
structure_files, log_files = upload_and_extract()

if structure_files:
    # 파일 선택 버튼 생성
    file_buttons = []
    for i, file_path in enumerate(structure_files):
        file_name = os.path.basename(file_path)
        button = widgets.Button(
            description=file_name[:20] + ('...' if len(file_name) > 20 else ''),
            tooltip=file_name,
            layout=widgets.Layout(width='auto', height='auto')
        )
        # 더 진한 회색으로 변경
        button.style.button_color = '#A0A0A0'  # 진한 회색
        button.style.text_color = 'black'
        button.file_path = file_path  # 파일 경로 저장
        file_buttons.append(button)

    # 스타일 선택 버튼 생성
    style_buttons = []
    # DiffDock은 리간드-단백질 복합체이므로 리간드 강조 스타일 추가
    styles = ['cartoon', 'ribbon', 'stick', 'sphere', 'line', 'surface', 'ligand_highlight']
    style_colors = {
        'cartoon': '#4682B4',    # 스틸 블루
        'ribbon': '#556B2F',     # 다크 올리브 그린
        'stick': '#8B4513',      # 새들 브라운
        'sphere': '#6A5ACD',     # 슬레이트 블루
        'line': '#2F4F4F',       # 다크 슬레이트 그레이
        'surface': '#B8860B',    # 다크 골든로드
        'ligand_highlight': '#FF6347'  # 토마토 색상
    }

    for style in styles:
        button = widgets.Button(
            description=style,
            layout=widgets.Layout(width='110px', height='auto')
        )
        button.style.button_color = style_colors[style]
        button.style.text_color = 'white'  # 어두운 배경에 흰색 글씨
        button.style_name = style  # 스타일 이름 저장
        style_buttons.append(button)

    # 로그 파일 버튼 생성
    log_buttons = []
    if log_files:
        for log_path in log_files:
            log_name = os.path.basename(log_path)
            button = widgets.Button(
                description=log_name[:20] + ('...' if len(log_name) > 20 else ''),
                tooltip=log_name,
                layout=widgets.Layout(width='auto', height='auto')
            )
            button.style.button_color = '#708090'  # 슬레이트 그레이
            button.style.text_color = 'white'
            button.log_path = log_path  # 로그 파일 경로 저장
            log_buttons.append(button)

    # 현재 선택된 파일과 스타일 저장
    state = {'current_file': structure_files[0], 'current_style': 'cartoon'}

    # 로그 출력 영역
    log_output = widgets.Output()

    # 버튼 클릭 이벤트 핸들러
    def on_file_button_clicked(b):
        state['current_file'] = b.file_path
        # 모든 파일 버튼 색상 초기화
        for button in file_buttons:
            button.style.button_color = '#A0A0A0'  # 진한 회색
            button.style.text_color = 'black'
        # 선택된 버튼 색상 변경
        b.style.button_color = '#228B22'  # 포레스트 그린 (더 진한 녹색)
        b.style.text_color = 'white'
        # 구조 시각화
        visualize_structure(state['current_file'], state['current_style'])
        # 버튼 다시 표시
        display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
        display(widgets.HBox(file_buttons))
        display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
        display(widgets.HBox(style_buttons))
        if log_buttons:
            display(widgets.HTML("<h3>로그 파일 선택:</h3>"))
            display(widgets.HBox(log_buttons))
            display(log_output)

    def on_style_button_clicked(b):
        state['current_style'] = b.style_name
        # 모든 스타일 버튼 색상 변경 (각 버튼마다 고유한 색상)
        for button in style_buttons:
            style = button.style_name
            if style == state['current_style']:
                # 선택된 버튼은 더 밝은 버전의 색상
                brightness_factor = 0.7
                r = int(int(style_colors[style][1:3], 16) * (1 + brightness_factor))
                g = int(int(style_colors[style][3:5], 16) * (1 + brightness_factor))
                b = int(int(style_colors[style][5:7], 16) * (1 + brightness_factor))
                r = min(r, 255)
                g = min(g, 255)
                b = min(b, 255)
                button.style.button_color = f'#{r:02x}{g:02x}{b:02x}'
            else:
                button.style.button_color = style_colors[style]
            button.style.text_color = 'white'

        # 구조 시각화
        visualize_structure(state['current_file'], state['current_style'])
        # 버튼 다시 표시
        display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
        display(widgets.HBox(file_buttons))
        display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
        display(widgets.HBox(style_buttons))
        if log_buttons:
            display(widgets.HTML("<h3>로그 파일 선택:</h3>"))
            display(widgets.HBox(log_buttons))
            display(log_output)

    def on_log_button_clicked(b):
        # 모든 로그 버튼 색상 초기화
        for button in log_buttons:
            button.style.button_color = '#708090'  # 슬레이트 그레이
        # 선택된 버튼 색상 변경
        b.style.button_color = '#4B0082'  # 인디고

        # 로그 파일 내용 표시
        log_output.clear_output()
        with log_output:
            content = view_log_file(b.log_path)
            if content:
                display(content)

    # 버튼에 이벤트 핸들러 연결
    for button in file_buttons:
        button.on_click(on_file_button_clicked)

    for button in style_buttons:
        button.on_click(on_style_button_clicked)

    for button in log_buttons:
        button.on_click(on_log_button_clicked)

    # 첫 번째 파일 자동 표시
    visualize_structure(structure_files[0], 'cartoon')

    # 버튼 표시
    display(widgets.HTML("<h3>구조 파일 선택:</h3>"))
    display(widgets.HBox(file_buttons))
    display(widgets.HTML("<h3>시각화 스타일 선택:</h3>"))
    display(widgets.HBox(style_buttons))

    if log_buttons:
        display(widgets.HTML("<h3>로그 파일 선택:</h3>"))
        display(widgets.HBox(log_buttons))
        display(log_output)

    # 첫 번째 파일 버튼 하이라이트
    file_buttons[0].style.button_color = '#228B22'  # 포레스트 그린
    file_buttons[0].style.text_color = 'white'

    # 카툰 스타일 버튼 하이라이트
    brightness_factor = 0.7
    r = int(int(style_colors['cartoon'][1:3], 16) * (1 + brightness_factor))
    g = int(int(style_colors['cartoon'][3:5], 16) * (1 + brightness_factor))
    b = int(int(style_colors['cartoon'][5:7], 16) * (1 + brightness_factor))
    r = min(r, 255)
    g = min(g, 255)
    b = min(b, 255)
    style_buttons[0].style.button_color = f'#{r:02x}{g:02x}{b:02x}'
    style_buttons[0].style.text_color = 'white'

    # 데이터 정보 표시
    print("\nDiffDock 결과 데이터 정보:")
    print(f"- 발견된 복합체 구조 파일: {len(structure_files)}개")
    print(f"- 발견된 로그 파일: {len(log_files)}개")

    # 간단한 사용 안내
    print("\n사용 안내:")
    print("- 구조 파일 버튼을 클릭하여 다른 결과를 확인할 수 있습니다.")
    print("- 스타일 버튼을 클릭하여 시각화 방식을 변경할 수 있습니다.")
    print("- 'ligand_highlight' 스타일은 리간드(약물)를 강조해서 보여줍니다.")
    if log_buttons:
        print("- 로그 파일 버튼을 클릭하여 DiffDock 결과 로그를 확인할 수 있습니다.")

파일 로딩 중: 6w70.pdb, 스타일: ligand_highlight


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

None


구조 시각화 완료: 6w70.pdb
파일이나 스타일을 변경하려면 아래 버튼을 사용하세요.


HTML(value='<h3>구조 파일 선택:</h3>')

HTML(value='<h3>시각화 스타일 선택:</h3>')

HTML(value='<h3>로그 파일 선택:</h3>')

Output()